# Importing required packages

In [1]:
import os
import re
from flask import Flask, request
from slack_sdk import WebClient
from slack_bolt import App
from slack_bolt.adapter.flask import SlackRequestHandler
import pandas as pd
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import find_dotenv, load_dotenv
from serpapi import GoogleSearch
import wikipedia

# Load environment variables

In [2]:
load_dotenv(find_dotenv())

True

# Set Slack API credentials

In [3]:
SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]

# Initialize the Slack app and Flask app only once

In [4]:
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

# OpenAI-based language model with conversational capabilities

In [5]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Create a memory buffer for maintaining context in conversations

In [6]:
memory = ConversationBufferMemory(memory_key="conversation", return_messages=True, k=20)

# Create system and human prompts

In [7]:
system_template = """
You are Alphie, a highly capable AI assistant.
Your main roles are to:
- Answer user questions with detailed and accurate information.
- Assist with various tasks, including programming, debugging, writing, data analysis, and more.
- Provide suggestions for personal and professional development.
- Help with productivity, organization, and automation.
- Engage in meaningful conversations, maintaining context to ensure continuity.

Key characteristics:
- Be friendly, helpful, and responsive.
- Offer clear explanations and useful advice.
- When necessary, reference sources to increase reliability and transparency.
- Guide users to appropriate resources or solutions, and be clear about any limitations.

Specific guidance:
- For technical queries, aim to be thorough and offer code examples when appropriate.
- For general knowledge, be informative and concise.
- For questions about specific documents or content, extract relevant information and mention sources.
- When unsure, ask users for clarification or suggest additional resources.

Your mission is to create a seamless user experience, helping users achieve their goals while ensuring a clear and supportive communication style.
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "User says: {user_input}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


# Conversational chain with memory

In [8]:
chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


# Installing & importing functions

In [9]:
!pip install functions

from functions import extract_text_from_pdf #1
from functions import clean_text #2
from functions import analyze_pdf #3
from functions import get_pdf_path_from_keyword #4
from functions import google_search #5
from functions import get_wikipedia_summary #6



# Load the mapping from the CSV file for keyword-to-PDF mapping

In [10]:
csv_path = "C:/Users/asifr/OneDrive - State of New Mexico\Documents/GitHub/langchain-experiments/slack/output_tables/article_titles.csv"  # Update with correct path
pdf_dir = "articles_output"  # Directory with PDFs

article_pdf_map = {}
df = pd.read_csv(csv_path)

# Populate the dictionary with ARTICLE numbers and short titles

In [11]:
for _, row in df.iterrows():
    article_number = row["ARTICLE Number"]
    short_title = row["Short Title"]

    pdf_name = f"ARTICLE {article_number}.pdf"
    pdf_path = os.path.join(pdf_dir, pdf_name)

    article_pdf_map[article_number] = pdf_path
    article_pdf_map[short_title.lower()] = pdf_path

# Slack event handler for message events

In [12]:
@app.event("message")
def handle_message_events(event, say):
    user_text = event.get("text", "").lower()

    # Respond to greetings
    if user_text in ["hi", "hello", "hey"]:
        say("Hello! I'm Alphie, your friendly AI assistant at the New Mexico Tax & Rev. How can I help you today?")
        return

    # Google Search functionality
    if "search" in user_text:
        query = user_text.replace("search", "").strip()
        search_results = google_search(query)

        results_text = "\n".join(
            [f"- {result['title']}: {result['link']}" for result in search_results.get("organic_results", [])[:3]]
        )
        say(f"Here are some search results for '{query}':\n{results_text}")
        return

    # Wikipedia search functionality
    if "what is" in user_text:
        topic = user_text.replace("what is", "").strip()
        wikipedia_summary = get_wikipedia_summary(topic)
        say(wikipedia_summary)
        return

    # Default response for other queries
    if pdf_path := get_pdf_path_from_keyword(user_text):
        question = re.sub(r"(" + "|".join(article_pdf_map.keys()) + ")", "", user_text).strip()
        response = analyze_pdf(pdf_path, question)

        if not response or "I don't know" in response:
            say("I'm not sure I understood that. Could you please rephrase your question?")
        else:
            say(response)
    else:
        response = chain.run({"user_input": user_text})
        say(response)

# Flask endpoint for Slack events

In [13]:
@flask_app.route("/slack/events", methods=["POST"])
def slack_events():
    return handler.handle(request)

In [14]:
# Initialize the Flask app

In [15]:
if __name__ in "__main__":
    flask_app.run(port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
